PILAR ARIAS, YEDRA MARTÍN, PATRICIA MEDIAVILLA, AMAYA SARASA

<GRUPO 9>

#Práctica 1

##Introducción

Esta es la primera práctica del segundo laboratorio. Dada una base de datos en formato .csv, se nos pide contestar a una serie de cuestiones. Para ello, lo priemro que debemos hacer es cargar y leer la base de datos:

In [55]:
#Importar librerías
#!apt install mlxtend
import numpy as np #numpy
import pandas as pd #pandas
import sklearn as sk #scikit-learn
import matplotlib.pyplot as plt #gráfica Matplotlib
#from mlxtend.frequent_patterns import apriori
#from apyori import apriori #apriori
from mlxtend.preprocessing import TransactionEncoder
#from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import apriori, association_rules
#from mlxtend.frequent_patterns import frequent_patterns
#from mlxtend import frequent_patterns
#from mlxtend.frequent_patterns import frequent_patterns

#Importamos la base de datos
df = pd.read_csv('BlackFriday.csv',sep=',')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233599 entries, 0 to 233598
Data columns (total 11 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   User_ID                     233599 non-null  int64  
 1   Product_ID                  233599 non-null  object 
 2   Gender                      233599 non-null  object 
 3   Age                         233599 non-null  object 
 4   Occupation                  233599 non-null  int64  
 5   City_Category               233599 non-null  object 
 6   Stay_In_Current_City_Years  233599 non-null  object 
 7   Marital_Status              233599 non-null  int64  
 8   Product_Category_1          233599 non-null  int64  
 9   Product_Category_2          161255 non-null  float64
 10  Product_Category_3          71037 non-null   float64
dtypes: float64(2), int64(4), object(5)
memory usage: 19.6+ MB


,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3
0,1000004,P00128942,M,46-50,7,B,2,1,1,11.0,NaN
1,1000009,P00113442,M,26-35,17,C,0,0,3,5.0,NaN
2,1000010,P00288442,F,36-45,1,B,4+,1,5,14.0,NaN
3,1000010,P00145342,F,36-45,1,B,4+,1,4,9.0,NaN
4,1000011,P00053842,F,26-35,1,C,1,0,4,5.0,12.0


----------------

----------------

Subimos la base de datos, df.info nos aporta la infomación que contiene la base de datos, vemos el tipo, la cantidad de las variables con las que trabajamos... y .head vamos a ver que forma tiene la base de datos y las primeras filas de esta.

Según se ha estudiado en clase, encontramos dos métodos para afrontarlo. Nos decantaremos por CRISP-DM. Las siglas para: CRoss-Industry Standard Process for Data Mining. 

Es un modelo de proceso de minería de datos que se utiliza en la industria. Es un marco de trabajo que define un proceso estandarizado para llevar a cabo proyectos de minería de datos, desde la definición del problema hasta la implementación y evaluación del resultado. Los seis pasos que componen Crisp-DM son los siguientes:

- Comprensión de los negocios y definición del problema: comprender la naturaleza del problema y los objetivos de negocio.

- Comprensión de los datos: recopilación inicial, descripción y exploración  de datos.

- Preparación de los datos: Selección, limpieza, construcción, integración, formateo de los datos.

- Modelado: aplicar las técnicas de minería de datos a los dataset.

- Evaluación: determinar si los resultados son útiles a las necesidades del negocio.

- Despliegue: explotar utilidad de los modelos, integrándolos en las tareas de toma de decisiones de la organización -> Call to Action. 

##Comprensión de los negocios y definición del problema

La librería se usará de la siguiente manera:
1) Empezaremos obteniendo los itemsets frecuentes para k=1. En este punto necesitaremos obtener el
soporte de los itemset. Por lo tanto, se tendrá que usar un método que dado un itemset devuelva su
soporte.

2) Para k≥2.
a. Mostrar los itemset frecuentes candidatos y su soporte.

3) Repetir el proceso 2 hasta que no se generen nuevos itemsets frecuentes.

4) Mostrar todas las posibles reglas con la confianza de cada una de ellas.

5) Listar todas las reglas que sean de alta confianza.

6) Usar los siguientes métodos. Dado un antecesor, devolver todas las reglas que contengan a dicho
antecesor. Dado un umbral mínimo devolver todas las reglas que cumplan con dicha confianza.

7) Utiliza las representaciones gráficas que consideres adecuadas para representar las reglas obtenidas
y obtener conclusiones a partir de los datos.

A partir de lo anterior de pide:
- Prueba al menos tres configuraciones de soporte y frecuencia para cada género, edad y tipo de producto.
- ¿Qué diferencias hay entre usar soporte y frecuencia? Respalda la respuesta con datos 
- ¿Qué tipo reglas desaparecen según la configuración y categorías (género, edad y tipo de producto)
consideradas? ¿Por qué? 
- Para una de las configuraciones, interpreta algunas de las reglas que te hayan resultado interesantes
obtenidas usando la clase association_rules y un par de configuraciones cambiando la variable metric y
min_threshold. Justifica los resultados. 
- Dadas las mejores configuraciones. ¿Existen reglas o patrones que se repitan? ¿Podemos generalizar de
alguna manera como se comportan los clientes? Expón las conclusiones respaldadas con datos 

## Comprensión de los datos

La base de datos que utilizaremos consta de 233599 observaciones. Teniendo en ceunta que el trabajo que vamos a realizar, utilizanod el algoritmo apriori, queremos sacar regalas y con todos estos datos, lo que nos hace pensar es que vamos a encontrar muchísimas reglas a partir de estos datos.

La clave está en cuales son las que más nos interesan, basandonos en su confianza, y una vez hallado esto debemos tomar decisiones para que usando estas reglas vendamos el mayor número de producto, para aumentar nuestras gananzas.

Existen 11 tipos de atributos distintos: 

- User_ID: número para identificar al usuario.
- Product_ID: número para identificar el producto.
- Gender: Género del comprador.	
- Age: Edad del comprador.
- Occupation: A lo que se dedica el comprador, ya pasasdo a categórica.
- City_Category: Categoría de la ciudad donde reside el user.
- Stay_In_Current_City_Years: El tiempo que lleva en la ciudad.
- Marital_Status: Estado civil.
- Product_Category_1: La categoría principal del producto.
- Product_Category_2: Siguente categoría.
- Product_Category_3: Última categoría del producto.



##Preparación de los datos

In [56]:
usernan = df['User_ID'].isna().sum()
print(f"La cantidad de NaN que existen en la variable User_ID es:{usernan}")
productnan = df['Product_ID'].isna().sum()
print(f"La cantidad de NaN que existen en la variable Product_ID es:{productnan}")
gendernan = df['Gender'].isna().sum()
print(f"La cantidad de NaN que existen en la variable Gender es:{gendernan}")
agenan = df['Age'].isna().sum()
print(f"La cantidad de NaN que existen en la variable Age es:{agenan}")
ocupationan = df['Occupation'].isna().sum()
print(f"La cantidad de NaN que existen en la variable Occupation es:{ocupationan}")
City_Categorynan = df['City_Category'].isna().sum()
print(f"La cantidad de NaN que existen en la variable City_Category es:{City_Categorynan}")
Stay_In_Current_City_Yearsnan = df['Stay_In_Current_City_Years'].isna().sum()
print(f"La cantidad de NaN que existen en la variable Stay_In_Current_City_Years es:{Stay_In_Current_City_Yearsnan}")
Marital_Statusnan = df['Marital_Status'].isna().sum()
print(f"La cantidad de NaN que existen en la variable Marital_Status es:{Marital_Statusnan}")
Product_Category_1nan = df['Product_Category_1'].isna().sum()
print(f"La cantidad de NaN que existen en la variable Product_Category_1 es:{Product_Category_1nan}")
Product_Category_2nan = df['Product_Category_2'].isna().sum()
print(f"La cantidad de NaN que existen en la variable Product_Category_2 es:{Product_Category_2nan}")
Product_Category_3nan = df['Product_Category_3'].isna().sum()
print(f"La cantidad de NaN que existen en la variable Product_Category_3 es:{Product_Category_3nan}")

La cantidad de NaN que existen en la variable User_ID es:0
La cantidad de NaN que existen en la variable Product_ID es:0
La cantidad de NaN que existen en la variable Gender es:0
La cantidad de NaN que existen en la variable Age es:0
La cantidad de NaN que existen en la variable Occupation es:0
La cantidad de NaN que existen en la variable City_Category es:0
La cantidad de NaN que existen en la variable Stay_In_Current_City_Years es:0
La cantidad de NaN que existen en la variable Marital_Status es:0
La cantidad de NaN que existen en la variable Product_Category_1 es:0
La cantidad de NaN que existen en la variable Product_Category_2 es:72344
La cantidad de NaN que existen en la variable Product_Category_3 es:162562


In [57]:
# Rellenamos los valores na con 0, ya que no tiene categoría
df = df.fillna(0)

# Quitamos los duplicados
df = df.drop_duplicates()

##Modelado

In [44]:
df_User_Product = (df
        .groupby(['User_ID', 'Product_ID'])['Product_Category_1']
        .sum()
        .apply(lambda x:x>0)
        .unstack()
        .fillna(False)
        .reset_index()
        .set_index('User_ID')
          )

In [45]:
df_Gender = (df
            .groupby(['User_ID','Gender'])['Product_Category_1']
            .sum()
            .apply(lambda x: x>0)
            .unstack()
            .fillna(False)
            .reset_index()
            .set_index('User_ID')
)

In [46]:
df_Age = (df
        .groupby(['User_ID','Age'])['Product_Category_1']
        .sum()
        .apply(lambda x: x>0)
        .unstack()
        .fillna(False)
        .reset_index()
        .set_index('User_ID')
)

In [47]:
df_Occupation = (df
                .groupby(['User_ID','Occupation'])['Product_Category_1']
                .sum()
                .apply(lambda x: x>0)
                .unstack()
                .fillna(False)
                .reset_index()
                .set_index('User_ID')
)

In [48]:
df_City = (df
          .groupby(['User_ID','City_Category'])['Product_Category_1']
          .sum()
          .apply(lambda x: x>0)
          .unstack()
          .fillna(False)
          .reset_index()
          .set_index('User_ID')
)

In [49]:
df_Stay = (df
           .groupby(['User_ID','Stay_In_Current_City_Years'])['Product_Category_1']
           .sum()
           .apply(lambda x: x>0)
           .unstack()
           .fillna(False)
           .reset_index()
           .set_index('User_ID')
)

In [50]:
df_Marital = (df
              .groupby(['User_ID','Marital_Status'])['Product_Category_1']
              .sum()
              .apply(lambda x: x>0)
              .unstack()
              .fillna(False)
              .reset_index()
              .set_index('User_ID')
)

In [51]:
df_Product_Category_1 = (df
                         .groupby(['User_ID','Product_Category_1'])['Product_Category_2']
                         .sum()
                         .apply(lambda x: x>0)
                         .unstack()
                         .fillna(False)
                         .reset_index()
                         .set_index('User_ID')
)

In [52]:
df_Product_Category_2 = (df
                         .groupby(['User_ID','Product_Category_2'])['Product_Category_1']
                         .sum()
                         .apply(lambda x: x>0)
                         .unstack()
                         .fillna(False)
                         .reset_index()
                         .set_index('User_ID')
)

In [53]:
df_Product_Category_3 = (df
                         .groupby(['User_ID','Product_Category_3'])['Product_Category_1']
                         .sum()
                         .apply(lambda x: x>0)
                         .unstack()
                         .fillna(False)
                         .reset_index()
                         .set_index('User_ID')
)

In [54]:
data = pd.concat([df_User_Product, df_Occupation, df_City, df_Stay, df_Marital, df_Product_Category_1, df_Product_Category_2, df_Product_Category_3, df_Age, df_Gender], axis = 1)
data

,P00000142,P00000242,P00000342,P00000442,P00000542,P00000642,P00000742,P00000842,P00000942,P00001042,...,18.0,0-17,18-25,26-35,36-45,46-50,51-55,55+,F,M
User_ID,,,,,,,,,,,,,,,,,,,,,
1000001,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,True,False
1000002,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,True
1000003,False,False,False,False,False,False,False,False,False,False,...,True,False,False,True,False,False,False,False,False,True
1000004,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,True
1000005,False,False,False,False,False,True,False,False,False,False,...,False,False,False,True,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1006036,False,False,False,False,False,False,False,False,False,False,...,True,False,False,True,False,False,False,False,True,False
1006037,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,True,False
1006038,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,True,False


In [58]:
print(data.shape)

(5891, 3583)


In [59]:
#Esto es para k=1 // use_colnames = True, max_len = 3
#frq_items = apriori(df, min_support = 0.05, use_colnames = True, max_len = 3)

frequent_itemsets = apriori(data, min_support=0.05, use_colnames=True, max_len=3)
frequent_itemsets

,support,itemsets
0,0.082159,(P00000142)
1,0.052792,(P00002142)
2,0.058903,(P00003242)
3,0.066033,(P00003442)
4,0.053641,(P00003942)
...,...,...
38085,0.074181,"(18-25, 17.0, M)"
38086,0.051774,"(17.0, 26-35, F)"
38087,0.150399,"(17.0, 26-35, M)"
38088,0.081310,"(17.0, 36-45, M)"


In [ ]:
type(frequent_itemsets)

pandas.core.frame.DataFrame

In [61]:
#Para k=2

frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets2=frequent_itemsets[frequent_itemsets['length'] == 3]
frequent_itemsets2.head()

,support,itemsets,length
3603,0.050416,"(0, P00000142, 3)",3
3604,0.050416,"(0, P00000142, 4.0)",3
3606,0.050416,"(0, P00000142, 5.0)",3
3607,0.058903,"(P00000142, 2, 1)",3
3608,0.077406,"(P00000142, 3, 1)",3


In [62]:
#Para k=3
frequent_itemsets3=frequent_itemsets[frequent_itemsets['length'] == 1]
frequent_itemsets3

,support,itemsets,length
0,0.082159,(P00000142),1
1,0.052792,(P00002142),1
2,0.058903,(P00003242),1
3,0.066033,(P00003442),1
4,0.053641,(P00003942),1
...,...,...,...
29464,0.392633,(8),1
30749,0.066372,(11.0),1
31199,0.057206,(12.0),1
31599,0.097946,(13.0),1


In [78]:
#Reglas de asociación
rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.8)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(P00000142),(0),0.082159,0.546936,0.079783,0.971074,1.775481,0.034847,15.663070
1,(P00000142),(1),0.082159,0.378713,0.077406,0.942149,2.487763,0.046291,10.739385
2,(P00000142),(3),0.082159,0.130708,0.082159,1.000000,7.650649,0.071420,inf
3,(P00000142),(4.0),0.082159,0.074181,0.082159,1.000000,13.480549,0.076065,inf
4,(4.0),(P00000142),0.074181,0.082159,0.082159,1.107551,13.480549,0.076065,inf
...,...,...,...,...,...,...,...,...,...
17383,"(17.0, F)",(9.0),0.069767,0.187065,0.065354,0.936740,5.007562,0.052303,12.850626
17384,"(9.0, 18.0)",(26-35),0.062977,0.348498,0.060431,0.959569,2.753443,0.038484,16.113823
17385,"(26-35, 18.0)",(M),0.085215,0.717196,0.070616,0.828685,1.155452,0.009501,1.650788
17386,"(18.0, M)",(26-35),0.070277,0.348498,0.070616,1.004831,2.883321,0.046125,inf


Pero tenemos que quitar aquellas reglas que tengan un 0 en el antecedente o en la consecuencia. 

In [68]:
mask = (rules['antecedents'].apply(lambda x: 0 in x) | rules['consequents'].apply(lambda x: 0 in x))

rules_with_zero = rules[mask]

In [69]:
rules = rules.drop(rules_with_zero.index)

In [79]:
#vemos reglas de alta confianza
rulesaltaconfianza = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.95)
rulesaltaconfianza

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(P00000142),(0),0.082159,0.546936,0.079783,0.971074,1.775481,0.034847,15.663070
1,(P00000142),(3),0.082159,0.130708,0.082159,1.000000,7.650649,0.071420,inf
2,(P00000142),(4.0),0.082159,0.074181,0.082159,1.000000,13.480549,0.076065,inf
3,(4.0),(P00000142),0.074181,0.082159,0.082159,1.107551,13.480549,0.076065,inf
4,(P00002142),(1),0.052792,0.378713,0.052792,1.000000,2.640520,0.032799,inf
...,...,...,...,...,...,...,...,...,...
10903,"(17.0, 36-45)",(M),0.067051,0.717196,0.081310,1.212658,1.690833,0.033221,inf
10904,"(17.0, 46-50)",(0.0),0.051265,0.546936,0.051265,1.000000,1.828367,0.023226,inf
10905,(12.0),"(9.0, F)",0.057206,0.098455,0.055169,0.964392,9.795227,0.049537,25.318381
10906,"(9.0, 18.0)",(26-35),0.062977,0.348498,0.060431,0.959569,2.753443,0.038484,16.113823


COPIO CÓDIGO QUE ES INTERESANTE PARA FUTURO


In [ ]:
#frequent_itemsets = apriori(data, min_support=0.6, use_colnames=True)
#frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
#frequent_itemsets1=frequent_itemsets[frequent_itemsets['length'] == 1]

In [ ]:
#frequent_itemsets[ (frequent_itemsets['length'] >1 ) & #esto sería para k=2
                   #(frequent_itemsets['support'] >= 0.8) ]

,support,itemsets,length


##Evaluación

##Despliegue